# Apply Low pass filter on drifters

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj

# from rasterio.transform import Affine

import matplotlib.pyplot as plt
import matplotlib.colors as cl

import hvplot.xarray

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo

crs = ccrs.PlateCarree()
import cmocean.cm as cm

# from xgcm import Grid
from xhistogram.xarray import histogram
import warnings

warnings.filterwarnings("ignore")

import os
from glob import glob

from histlib.cstes import zarr_dir 

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    cluster = PBSCluster(cores=7, processes=7, walltime="04:00:00")
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster

    #
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.62:8787/status,
Dashboard: http://10.148.1.62:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.62:43751,Workers: 0
Dashboard: http://10.148.1.62:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
def add_ggx_attrs(ds_data):
    listv = [l for l in list(ds_data.variables) if 'sla' in l]+['alti_mdt','alti_ocean_tide', 'alti_dac', 'alti_internal_tide']
    listv = [l for l in listv if 'gg' not in l]
    for v in listv :
        ds_data[v.replace('alti', 'alti_ggx')].attrs['comment'] = ds_data[v].attrs['comment']
        ds_data[v.replace('alti', 'alti_ggx')].attrs['units'] = r'$m.s^{-2}$'
        ds_data[v.replace('alti', 'alti_ggx')].attrs['long_name']= r'$g\partial_x$'+v.replace('alti_','')

def add_adt_to_ds_data(ds_data):
    add_ggx_attrs(ds_data)
    ds_data = ds_data.rename({'drifter_acc_x':'drifter_acc_x_0', 'drifter_acc_y':'drifter_acc_y_0', 'drifter_coriolis_x':'drifter_coriolis_x_0', 'drifter_coriolis_y':'drifter_coriolis_y_0'})
    for sla in ['alti_ggx_sla_filtered','alti_ggx_sla_unfiltered','alti_ggx_sla_unfiltered_denoised','alti_ggx_sla_unfiltered_imf1']:
        ds_data[sla.replace('sla', 'adt')] = ds_data[sla] + ds_data.alti_ggx_mdt
        ds_data[sla.replace('sla', 'adt')].attrs['comment'] = ds_data[sla].attrs['comment']
        ds_data[sla.replace('sla', 'adt')].attrs['units'] = r'$m.s^{-2}$'
        ds_data[sla.replace('sla', 'adt')].attrs['long_name']= ds_data[sla].attrs['long_name'].replace('sla', 'adt')
    return ds_data

In [20]:
# create 
ds = xr.open_zarr(os.path.join(zarr_dir,'test', 'gps_Jason-3_2020.zarr')).chunk({'obs':5}).persist()
ds = add_adt_to_ds_data(ds)

In [21]:
def add_low_pass_filter_to_data(ds, T=2, cutoff=4) : 
    from scipy.signal import filtfilt
    from scipy.integrate import cumulative_trapezoid
    from scipy.optimize import minimize
    # coefficients
    dt = (ds.drifter_time.diff('site_obs')/pd.Timedelta('1D')).mean()  # in days
    from pynsitu.tseries import generate_filter

    taps = generate_filter(band="low", dt=dt, T=T, bandwidth=cutoff)
        # apply filter
    #ds[f'drifter_ve_{cutoff}'] = xr.DataArray(filtfilt(taps, 1, ds.drifter_ve), dims=['obs', 'site_obs']).assign_attrs(**ds.drifter_ve.attrs)
    #ds[f'drifter_vn_{cutoff}'] = xr.DataArray(filtfilt(taps, 1, ds.drifter_vn), dims=['obs', 'site_obs']).assign_attrs(**ds.drifter_vn.attrs)
    #ds[f'drifter_ve_{cutoff}'].attrs['description'] = ds[f'drifter_ve_{cutoff}'].attrs['description'] + f' filtered with {cutoff} cpd cutoff frequency'
    #ds[f'drifter_vn_{cutoff}'].attrs['description'] = ds[f'drifter_vn_{cutoff}'].attrs['description'] + f' filtered with {cutoff} cpd cutoff frequency'
    vx = xr.DataArray(filtfilt(taps, 1, ds.drifter_vx), dims=['obs', 'site_obs'])
    vy = xr.DataArray(filtfilt(taps, 1, ds.drifter_vy), dims=['obs', 'site_obs'])
    ds[f"drifter_acc_x_{cutoff}"] = (vx.differentiate("site_obs")/3600).assign_attrs(**ds.drifter_acc_x_0.attrs).assign_attrs(description= ds.drifter_acc_x_0.attrs['description'] + f' filtered with {cutoff} cpd frequency',cutoff=cutoff)
    ds[f"drifter_acc_y_{cutoff}"] = (vy.differentiate("site_obs")/3600).assign_attrs(**ds.drifter_acc_x_0.attrs).assign_attrs(description= ds.drifter_acc_y_0.attrs['description'] + f' filtered with {cutoff} cpd frequency',cutoff=cutoff)
    ds[f"drifter_coriolis_x_{cutoff}"] = (-vy * ds.f).assign_attrs(ds.drifter_coriolis_x_0.attrs).assign_attrs(description= ds.drifter_coriolis_x_0.attrs['description'] + f' filtered with {cutoff} cpd frequency',cutoff=cutoff)
    ds[f"drifter_coriolis_y_{cutoff}"] = (vx * ds.f).assign_attrs(ds.drifter_coriolis_y_0.attrs).assign_attrs(description= ds.drifter_coriolis_y_0.attrs['description'] + f' filtered with {cutoff} cpd frequency',cutoff=cutoff)
    
    return ds

In [22]:
ds = add_low_pass_filter_to_data(ds, T=20, cutoff=1)
ds

In [25]:
i=0
ds_ = ds.isel(obs=i)
ds_ = ds_.reset_coords(['drifter_x', 'drifter_y'])
ds_.drifter_acc_x_0.hvplot(kind='scatter', label='no filter')*ds_.drifter_acc_x_1.hvplot(kind='scatter', label= 'cutoff=1')

:Overlay
   .Scatter.No_filter       :Scatter   [drifter_time]   (drifter_acc_x_0)
   .Scatter.Cutoff_equals_1 :Scatter   [drifter_time]   (drifter_acc_x_1)

In [27]:
cluster.close()